In [64]:
import pandas as pd
import requests

In [65]:
# From tutorial: http://www.bd-econ.com/imfapi1.html

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.JP.PMP_IX.?startPeriod=1999' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

baseyr = data['@BASE_YEAR']  # Save the base year

# Create pandas dataframe from the observations
data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')]
             for obs in data['Obs']]

df = pd.DataFrame(data_list, columns=['date', 'value'])
     
df = df.set_index(pd.to_datetime(df['date']))['value'].astype('float')
df

date
1999-01-01    73.830237
1999-02-01    74.676699
1999-03-01    75.335058
1999-04-01    75.993416
1999-05-01    77.780390
                ...    
2021-06-01    85.586645
2021-07-01    86.150952
2021-08-01    86.621209
2021-09-01    86.715260
2021-10-01    86.527157
Name: value, Length: 267, dtype: float64

In [66]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  # Method with series information
search_term = 'Trade'  # Term to find in series names
series_list = requests.get(f'{url}{key}').json()\
            ['Structure']['Dataflows']['Dataflow']

# Use dict keys to navigate through results:
for series in series_list:
    if search_term in series['Name']['#text']:
        print(f"{series['Name']['#text']}: {series['KeyFamilyRef']['KeyFamilyID']}")

Direction of Trade Statistics (DOTS), 2020 Q1: DOT_2020Q1
Direction of Trade Statistics (DOTS), 2018 Q3: DOT_2018Q3
Direction of Trade Statistics (DOTS), 2018 Q2: DOT_2018Q2
Direction of Trade Statistics (DOTS), 2019 Q1: DOT_2019Q1
Direction of Trade Statistics (DOTS), 2017 Q2: DOT_2017Q2
Direction of Trade Statistics (DOTS), 2018 Q1: DOT_2018Q1
Direction of Trade Statistics (DOTS), 2019 Q4: DOT_2019Q4
Direction of Trade Statistics (DOTS), 2017 Q4: DOT_2017Q4
Direction of Trade Statistics (DOTS), 2019 Q2: DOT_2019Q2
Direction of Trade Statistics (DOTS), 2020 Q2: DOT_2020Q2
Direction of Trade Statistics (DOTS), 2018 Q4: DOT_2018Q4
Direction of Trade Statistics (DOTS), 2019 Q3: DOT_2019Q3
Direction of Trade Statistics (DOTS), 2017 Q3: DOT_2017Q3
Direction of Trade Statistics (DOTS), 2017 Q1: DOT_2017Q1
Direction of Trade Statistics (DOTS), 2020 Q4: DOT_2020Q4
Direction of Trade Statistics (DOTS), 2020 Q3: DOT_2020Q3
Direction of Trade Statistics (DOTS), 2021 Q1: DOT_2021Q1
Direction of T

In [67]:
key = 'DataStructure/DOT'  # Method / series
dimension_list = requests.get(f'{url}{key}').json()['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f'Dimension {n+1}: {dimension["@codelist"]}')

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_DOT
Dimension 3: CL_INDICATOR_DOT
Dimension 4: CL_COUNTERPART_AREA_DOT


In [68]:
# Example: codes for third dimension, which is 2 in python
key = f"CodeList/{dimension_list[2]['@codelist']}"
code_list = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list:
    print(f"{code['Description']['#text']}: {code['@value']}")

Goods, Value of Exports, Free on board (FOB), US Dollars: TXG_FOB_USD
Goods, Value of Imports, Cost, Insurance, Freight (CIF), US Dollars: TMG_CIF_USD
Goods, Value of Imports, Free on board (FOB), US Dollars: TMG_FOB_USD
Goods, Value of Trade Balance, US Dollars: TBG_USD


## CSV import

Buld download from: https://data.imf.org/?sk=388dfa60-1d26-4ade-b505-a05a558d9a42&sid=1479329334655

In [69]:
unemployment = pd.read_csv('RawData/IFS_09-23-2023 15-20-12-31_timeSeries.csv')
unemployment[(unemployment['Country Name'] == 'Argentina') & (unemployment['Indicator Code'] == 'LU_PE_PC_CP_A_PT')].dropna(axis=1, thresh=1)

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,1971,1972,1973,1974,1975,...,2020Q1,2020Q2,2020Q3,2020Q4,2021,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1
472,Argentina,213,"Unemployment, Persons, Percentage change, corr...",LU_PE_PC_CP_A_PT,Value,24.367089,12.722646,-21.896163,-29.942197,-20.60231,...,NaN,NaN,NaN,NaN,-17.225271,NaN,NaN,NaN,NaN,NaN
473,Argentina,213,"Unemployment, Persons, Percentage change, corr...",LU_PE_PC_CP_A_PT,Value,NaN,NaN,NaN,NaN,NaN,...,4.161009,-0.065668,9.914268,18.59242,NaN,-2.795324,-11.42211,-21.30339,-33.221123,-30.507716
